In [8]:
import torch
from torch import nn

In [4]:
tensor1=torch.rand((3,3))
tensor2=torch.rand((3,3))

In [5]:
tensor1

tensor([[0.5134, 0.2053, 0.5163],
        [0.8513, 0.1882, 0.2458],
        [0.7266, 0.1816, 0.8688]])

In [6]:
tensor2

tensor([[0.7545, 0.4949, 0.2371],
        [0.9569, 0.0753, 0.2499],
        [0.6166, 0.7483, 0.3898]])

In [7]:
tensor1*tensor2

tensor([[0.3873, 0.1016, 0.1224],
        [0.8146, 0.0142, 0.0614],
        [0.4481, 0.1359, 0.3387]])

#### LTR => Long-Term to remember
#### STM => Short Term Memory
#### LTM => Long Term Memory

In [86]:
class NDIM_LSTM(nn.Module):
    
    def __init__(self,size):
        super().__init__()

        self.percent_ltr_input=nn.Parameter(torch.empty(size).normal_(mean=0.0,std=1.0),requires_grad=True)
        self.percent_ltr_stm_wt=nn.Parameter(torch.empty(size).normal_(mean=0.0,std=1.0),requires_grad=True)
        self.b1=nn.Parameter(torch.tensor(0.),requires_grad=False)

        self.percent_potential_ltm_stm_wt=nn.Parameter(torch.empty(size).normal_(mean=0.0,std=1.0),requires_grad=True)
        self.percent_potential_ltm_input=nn.Parameter(torch.empty(size).normal_(mean=0.0,std=1.0),requires_grad=True)
        self.b2=nn.Parameter(torch.tensor(0.),requires_grad=False)
        
        self.potential_ltm_stm_wt=nn.Parameter(torch.empty(size).normal_(mean=0.0,std=1.0),requires_grad=True)
        self.potential_ltm_input=nn.Parameter(torch.empty(size).normal_(mean=0.0,std=1.0),requires_grad=True)
        self.b3=nn.Parameter(torch.tensor(0.),requires_grad=False)
        
        self.output_stm_contri_stm_wt=nn.Parameter(torch.empty(size).normal_(mean=0.0,std=1.0),requires_grad=True)
        self.output_stm_contri_input=nn.Parameter(torch.empty(size).normal_(mean=0.0,std=1.0),requires_grad=True)
        self.b4=nn.Parameter(torch.tensor(0.),requires_grad=False)

    def lstm_unit(self,input_value,long_memory,short_memory):
        
        long_remember_percent=torch.sigmoid((input_value*self.percent_ltr_input)+
                                            (self.percent_ltr_stm_wt*short_memory)+
                                            self.b1)
        
        potential_remember_percent=torch.sigmoid((input_value*self.percent_potential_ltm_input)+
                                                 (short_memory*self.percent_potential_ltm_stm_wt)+
                                                  self.b2)

        potential_memory = torch.tanh((short_memory * self.potential_ltm_stm_wt) + 
                                  (input_value * self.potential_ltm_input) + 
                                  self.b3)
        
        updated_long_memory = ((long_memory * long_remember_percent) + 
               (potential_remember_percent * potential_memory))

        output_percent = torch.sigmoid((short_memory * self.output_stm_contri_stm_wt) + 
                                       (input_value * self.output_stm_contri_input) + 
                                       self.b4)         
        
        updated_short_memory = torch.tanh(updated_long_memory) * output_percent

        return([updated_long_memory, updated_short_memory])

    def forward(self, input, long_memory=0, short_memory=0): 
        
        return self.lstm_unit(input,long_memory,short_memory)

In [94]:
input_test=torch.rand((5,64,64,64))

In [95]:
model=NDIM_LSTM((64,64,64))

In [96]:
ltm=0
stm=0
for i in input_test:
    ltm,stm=model(i,ltm,stm)
    print(stm.detach().shape, ltm.detach().shape)

torch.Size([64, 64, 64]) torch.Size([64, 64, 64])
torch.Size([64, 64, 64]) torch.Size([64, 64, 64])
torch.Size([64, 64, 64]) torch.Size([64, 64, 64])
torch.Size([64, 64, 64]) torch.Size([64, 64, 64])
torch.Size([64, 64, 64]) torch.Size([64, 64, 64])
